<a href="https://colab.research.google.com/github/SeungukJeong/ML4AI/blob/main/feature_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 


import numpy as np
import pandas as pd
import glob
import math 

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline 
from sklearn.tree import DecisionTreeRegressor 

from lightgbm import LGBMRegressor 
from xgboost import XGBRegressor 

#import optuna #Optuna: 하이퍼파라미터 최적화 프레임워크
#from optuna.samplers import TPESampler 

import scipy
import scipy.signal

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense 
from sklearn.model_selection import KFold 
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import os 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
project_path = '/content/drive/MyDrive/FinalProject'

os.chdir(project_path)
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1P8ce0awz1gepxsBj5WPzoWZSy8UT7Yku/FinalProject'

In [ ]:
train_time = pd.read_csv('train.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_list = os.listdir('/content/drive/MyDrive/FinalProject/train')
example = pd.read_csv('train/' + train_list[0])

In [ ]:
example[:5]

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
0,268.0,879.0,-32.0,218.0,-566.0,217.0,135.0,-933.0,1446.0,-57.0
1,157.0,1586.0,-36.0,342.0,-653.0,167.0,226.0,-672.0,1276.0,-78.0
2,138.0,2132.0,-21.0,439.0,-734.0,37.0,355.0,-535.0,1137.0,-127.0
3,103.0,2477.0,0.0,506.0,-784.0,-43.0,458.0,-685.0,1017.0,-194.0
4,12.0,2658.0,31.0,545.0,-805.0,118.0,511.0,-720.0,936.0,-270.0


In [ ]:
example.describe()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
count,60001.000000,60001.000000,60001.000000,60001.000000,60001.000000,60001.000000,60001.000000,60001.000000,58229.000000,60001.000000
mean,10.297245,0.402560,-1.329711,-2.898518,0.705788,15.375994,-16.563007,-2.899902,-10.276254,2.216696
std,1080.658741,1388.566153,1097.419061,1076.602492,374.755707,623.507169,583.053755,717.680114,1321.372397,1697.782379
min,-4329.000000,-6228.000000,-4042.000000,-4643.000000,-1511.000000,-2458.000000,-3453.000000,-6437.000000,-4868.000000,-6801.000000
25%,-699.000000,-909.000000,-742.000000,-723.000000,-251.000000,-401.000000,-404.000000,-471.000000,-919.000000,-1159.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,723.000000,908.000000,746.000000,723.000000,255.000000,431.000000,373.000000,480.000000,883.000000,1149.000000
max,4866.000000,9097.000000,3833.000000,4805.000000,1487.000000,2340.000000,3027.000000,5009.000000,4572.000000,7266.000000


In [ ]:
def create_frame(data, data_time=None, type_data='train'):
    data = data.fillna(0)
    
    data_transform = data.describe().iloc[1:, :] 

    data_transform.loc['skew'] = data.skew().tolist()
    data_transform.loc['mad'] = data.mad().tolist()
    data_transform.loc['kurtosis'] = data.kurtosis().tolist()
            
    data_transform = pd.DataFrame(data_transform.unstack()).reset_index()
    data_transform = data_transform.rename(columns={0: 'value'})
    data_transform['feature'] = data_transform['level_0'] + '_' + data_transform['level_1']
    data_transform = data_transform.drop(['level_0', 'level_1'], axis=1).set_index('feature').T
    
    if type_data=='train':
        data_transform['time'] = data_time
    return data_transform

In [ ]:
all_train = pd.DataFrame()

for file in tqdm(train_list):
    df = pd.read_csv('train/' + file)
    data_time = train_time[train_time.segment_id == int(file.split('.')[0])].time_to_eruption.values[0]
    df = create_frame(df, data_time, type_data='train')
    df['segment_id'] = file[:-4]
    all_train = all_train.append(df)

all_train = all_train.reset_index(drop=True)

100%|██████████| 4431/4431 [26:31<00:00,  2.78it/s]


In [ ]:
all_train

feature,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_25%,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,...,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max,sensor_10_skew,sensor_10_mad,sensor_10_kurtosis,time,segment_id
0,10.297245,1080.658741,-4329.0,-699.0,0.0,723.0,4866.0,0.021152,853.814395,0.202543,...,-6801.0,-1159.0,0.0,1149.0,7266.0,0.026519,1355.517274,0.068443,7169486,579683001
1,4.469659,804.238649,-3498.0,-532.0,0.0,552.0,3402.0,-0.005650,641.805415,0.052170,...,-5006.0,-911.0,0.0,935.0,6596.0,0.171276,1113.111210,0.171414,8297866,580262567
2,-1.726988,480.404868,-3785.0,-275.0,0.0,274.0,5152.0,-0.039636,354.969467,3.272190,...,-3184.0,-463.0,0.0,455.0,3206.0,-0.027107,558.263159,0.410877,27527640,579307667
3,0.964634,238.390899,-1971.0,-133.0,0.0,133.0,2318.0,0.094812,171.153800,6.870669,...,-5348.0,-285.0,0.0,291.0,5346.0,-0.116301,374.512975,8.387986,31603911,579908624
4,0.946784,287.084536,-1635.0,-161.0,0.0,165.0,2160.0,0.059142,212.300404,2.558708,...,-6185.0,-262.0,0.0,261.0,4411.0,-0.309104,360.005908,9.421108,37782140,580272496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,-0.212663,319.887610,-1832.0,-202.0,0.0,205.0,1615.0,-0.063950,248.648551,0.806302,...,-5037.0,-382.0,0.0,393.0,4673.0,-0.057124,487.623304,3.328008,33398381,1171869234
4427,-1.968251,443.659054,-2173.0,-283.0,0.0,282.0,2253.0,-0.054897,346.069170,0.585543,...,-5619.0,-643.0,0.0,650.0,5484.0,0.027990,796.166237,0.847839,21894333,1173524986
4428,-2.910151,256.075256,-1191.0,-173.0,0.0,168.0,977.0,-0.015206,203.261700,0.130361,...,-1966.0,-307.0,0.0,300.0,2234.0,-0.032894,369.368263,0.400108,10106188,1174567207
4429,-0.206263,149.034789,-922.0,-93.0,0.0,96.0,728.0,-0.070561,115.520364,0.813147,...,-2337.0,-179.0,0.0,175.0,2313.0,-0.039165,227.818875,2.995381,39304003,1174005524


In [ ]:
all_train.to_csv('training_stat_add.csv')

In [ ]:
#tit.drop(['age_by_10', 'family_no'], axis =1 , inplace = True)

In [ ]:
all_train.drop(['time'],axis=1,inplace=True)

In [ ]:
all_train.to_csv('training_ss.csv')

# 파일 합치기

In [ ]:
df_stat = pd.read_csv('/content/drive/MyDrive/FinalProject/Feature_data/data/training_ss.csv')
df_stft = pd.read_csv('/content/drive/MyDrive/FinalProject/Feature_data/data/stft.csv')

In [ ]:
df_stft

,segment_id,s1_A_pow,s1_A_num,s1_BH_pow,s1_BH_num,s1_BL_pow,s1_BL_num,s1_C_pow,s1_C_num,s1_D_pow,...,s10_A_pow,s10_A_num,s10_BH_pow,s10_BH_num,s10_BL_pow,s10_BL_num,s10_C_pow,s10_C_num,s10_D_pow,s10_D_num
0,1136037770,0.000000,0.0,5630.045792,271.0,46725.41373,841.0,37140.92555,452.0,72426.03253,...,2120.558313,43.0,81355.19078,1444.0,84886.27047,807.0,60996.87159,442.0,157529.5636,1825.0
1,1969647810,0.000000,0.0,8239.546856,261.0,96756.95475,905.0,66797.01982,459.0,155685.00260,...,0.000000,0.0,110965.70700,1173.0,201698.82820,894.0,106227.82270,446.0,408403.9037,2161.0
2,1895879680,71.725726,5.0,8870.920230,441.0,49992.75371,895.0,33186.92546,451.0,79895.06926,...,37.333602,1.0,103640.52290,1860.0,103565.33620,878.0,49367.07192,433.0,194035.6253,2035.0
3,2068207140,29.114857,2.0,6797.868928,388.0,54570.01793,893.0,31405.33885,460.0,80809.40532,...,247.300004,6.0,72740.36781,1260.0,117559.71330,886.0,50623.05161,425.0,228879.9843,2082.0
4,192955606,0.000000,0.0,6567.271540,308.0,57132.88321,892.0,23944.96514,441.0,104701.41200,...,0.000000,0.0,86168.97815,1310.0,121979.50700,862.0,38900.16413,406.0,256294.3095,2122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,873340274,0.000000,0.0,2978.744448,76.0,139442.56290,901.0,78804.28255,456.0,184170.23550,...,0.000000,0.0,79267.94514,825.0,248995.27580,903.0,133503.17960,454.0,437541.6113,2104.0
4427,1297437712,12124.863950,122.0,10834.912900,181.0,182842.71580,911.0,71091.89343,456.0,324133.08940,...,26249.875530,105.0,141169.13220,997.0,332693.09650,915.0,104824.62790,430.0,657478.4799,2207.0
4428,694853998,116.085819,14.0,5974.792454,487.0,24603.01676,887.0,11406.61581,434.0,44864.72209,...,225.658882,12.0,54157.10773,1417.0,53309.87121,871.0,17962.78185,404.0,101289.8225,2061.0
4429,1886987043,82185.532230,1776.0,5448.726206,182.0,60810.93609,837.0,38213.79871,438.0,105242.84510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_entropy = pd.read_csv('/content/drive/MyDrive/FinalProject/Feature_data/entropy_add.csv')

In [ ]:
df_entropy.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
df_entropy

,segment_id,time_to_eruption,sensor1_entropy,sensor2_entropy,sensor3_entropy,sensor4_entropy,sensor5_entropy,sensor6_entropy,sensor7_entropy,sensor8_entropy,sensor9_entropy,sensor10_entropy
0,1136037770,12262005,1.540608,1.040269,1.467118,1.505731,NaN,1.732328,1.439844,1.585071,1.447728,1.498953
1,1969647810,32739612,1.483172,1.119343,1.400873,1.356298,1.120719,1.454848,1.225308,1.581336,1.291452,1.386649
2,1895879680,14965999,1.517500,0.957624,1.375496,1.322392,NaN,1.765919,1.590624,1.777654,1.233879,1.399941
3,2068207140,26469720,1.424799,1.246503,1.344512,1.553285,0.991803,1.260535,1.395859,1.523730,1.253006,1.249022
4,192955606,31072429,1.576645,NaN,1.425069,1.466598,1.356568,1.709410,1.815007,1.738943,0.492986,1.414840
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,873340274,15695097,1.554222,1.176783,1.604323,1.510968,1.107251,1.758790,1.655342,1.591384,1.500730,1.397526
4427,1297437712,35659379,1.250907,NaN,1.112170,0.574308,NaN,1.421388,1.234295,1.048284,0.405077,0.752395
4428,694853998,31206935,1.301582,0.924384,1.509003,1.135806,0.848969,1.014093,0.468549,0.784372,1.163484,1.201563
4429,1886987043,9598270,1.045409,NaN,1.237076,1.146465,0.580837,1.777112,1.439062,NaN,1.253037,1.282653


In [ ]:
df_merge1 = pd.merge(df_stft, df_stat, on='segment_id', how='inner')

In [ ]:
df_merge1

,segment_id,s1_A_pow,s1_A_num,s1_BH_pow,s1_BH_num,s1_BL_pow,s1_BL_num,s1_C_pow,s1_C_num,s1_D_pow,...,sensor_10_mean,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max,sensor_10_skew,sensor_10_mad,sensor_10_kurtosis
0,1136037770,0.000000,0.0,5630.045792,271.0,46725.41373,841.0,37140.92555,452.0,72426.03253,...,2.599440,596.865818,-3527,-406,0,410,2187,-0.048443,477.586298,0.102101
1,1969647810,0.000000,0.0,8239.546856,261.0,96756.95475,905.0,66797.01982,459.0,155685.00260,...,19.895518,932.699662,-4591,-547,0,565,5319,0.202520,707.192910,1.607990
2,1895879680,71.725726,5.0,8870.920230,441.0,49992.75371,895.0,33186.92546,451.0,79895.06926,...,0.968101,500.544892,-2798,-323,0,321,2522,0.007192,391.105987,0.615849
3,2068207140,29.114857,2.0,6797.868928,388.0,54570.01793,893.0,31405.33885,460.0,80809.40532,...,8.381444,523.449843,-2911,-323,0,326,3614,0.229183,402.646486,1.409321
4,192955606,0.000000,0.0,6567.271540,308.0,57132.88321,892.0,23944.96514,441.0,104701.41200,...,-3.740471,562.748316,-2611,-362,0,361,3274,-0.014160,440.599622,0.483804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,873340274,0.000000,0.0,2978.744448,76.0,139442.56290,901.0,78804.28255,456.0,184170.23550,...,-3.377394,1053.539787,-5726,-674,0,665,5493,-0.001635,820.739743,0.565751
4427,1297437712,12124.863950,122.0,10834.912900,181.0,182842.71580,911.0,71091.89343,456.0,324133.08940,...,10.939634,1279.909391,-13017,-786,0,791,14873,0.108147,967.746987,5.551134
4428,694853998,116.085819,14.0,5974.792454,487.0,24603.01676,887.0,11406.61581,434.0,44864.72209,...,-1.550374,251.560198,-1708,-149,0,148,1549,-0.072757,188.760376,2.441965
4429,1886987043,82185.532230,1776.0,5448.726206,182.0,60810.93609,837.0,38213.79871,438.0,105242.84510,...,-5.925085,597.353672,-3588,-368,0,356,3628,-0.029609,449.236581,1.345414


In [ ]:
df_merge2 = pd.merge(df_merge1, df_entropy, on = 'segment_id',how='inner')

In [ ]:
df_merge2

,segment_id,s1_A_pow,s1_A_num,s1_BH_pow,s1_BH_num,s1_BL_pow,s1_BL_num,s1_C_pow,s1_C_num,s1_D_pow,...,sensor1_entropy,sensor2_entropy,sensor3_entropy,sensor4_entropy,sensor5_entropy,sensor6_entropy,sensor7_entropy,sensor8_entropy,sensor9_entropy,sensor10_entropy
0,1136037770,0.000000,0.0,5630.045792,271.0,46725.41373,841.0,37140.92555,452.0,72426.03253,...,1.540608,1.040269,1.467118,1.505731,NaN,1.732328,1.439844,1.585071,1.447728,1.498953
1,1969647810,0.000000,0.0,8239.546856,261.0,96756.95475,905.0,66797.01982,459.0,155685.00260,...,1.483172,1.119343,1.400873,1.356298,1.120719,1.454848,1.225308,1.581336,1.291452,1.386649
2,1895879680,71.725726,5.0,8870.920230,441.0,49992.75371,895.0,33186.92546,451.0,79895.06926,...,1.517500,0.957624,1.375496,1.322392,NaN,1.765919,1.590624,1.777654,1.233879,1.399941
3,2068207140,29.114857,2.0,6797.868928,388.0,54570.01793,893.0,31405.33885,460.0,80809.40532,...,1.424799,1.246503,1.344512,1.553285,0.991803,1.260535,1.395859,1.523730,1.253006,1.249022
4,192955606,0.000000,0.0,6567.271540,308.0,57132.88321,892.0,23944.96514,441.0,104701.41200,...,1.576645,NaN,1.425069,1.466598,1.356568,1.709410,1.815007,1.738943,0.492986,1.414840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,873340274,0.000000,0.0,2978.744448,76.0,139442.56290,901.0,78804.28255,456.0,184170.23550,...,1.554222,1.176783,1.604323,1.510968,1.107251,1.758790,1.655342,1.591384,1.500730,1.397526
4427,1297437712,12124.863950,122.0,10834.912900,181.0,182842.71580,911.0,71091.89343,456.0,324133.08940,...,1.250907,NaN,1.112170,0.574308,NaN,1.421388,1.234295,1.048284,0.405077,0.752395
4428,694853998,116.085819,14.0,5974.792454,487.0,24603.01676,887.0,11406.61581,434.0,44864.72209,...,1.301582,0.924384,1.509003,1.135806,0.848969,1.014093,0.468549,0.784372,1.163484,1.201563
4429,1886987043,82185.532230,1776.0,5448.726206,182.0,60810.93609,837.0,38213.79871,438.0,105242.84510,...,1.045409,NaN,1.237076,1.146465,0.580837,1.777112,1.439062,NaN,1.253037,1.282653


In [ ]:
df_merge2['time_to_eruption']

0       12262005
1       32739612
2       14965999
3       26469720
4       31072429
          ...   
4426    15695097
4427    35659379
4428    31206935
4429     9598270
4430    20128938
Name: time_to_eruption, Length: 4431, dtype: int64

In [ ]:
df_merge2.to_csv('df_merge.csv')